# 16 - Pontos de Interesse e casamento de imagens

Neste roteiro iremos estudar o comportamento de algoritmos de detecção e descrição de pontos de interesse e como usá-los para fazer uma versão simplificada de um algoritmo de busca de imagens. 

## AVISO

Existe uma grande quantidade de tutoriais online com códigos de exemplo para este problema, mas **seu uso não é recomendado**. Além de muitos conterem material claramente desatualizado, muitos tutoriais encorajam um simples copiar-colar de código sem levar em conta as *ideias que o motivaram*. Nesta atividade recomendamos usar, como material complementar, a [documentação do OpenCV em Python](https://docs.opencv.org/3.4.3/d6/d00/tutorial_py_root.html) sobre [Feature Detection and Description](https://docs.opencv.org/3.4.3/db/d27/tutorial_py_table_of_contents_feature2d.html).

## Banco de imagens para a aula

Neste primeiro roteiro iremos trabalhar com um conjunto de imagens que não é focado exclusivamente em recuperação de informação, mas sim em detecção de objetos. Iremos usar a versão de 2005 do [Pascal VOC Challenge](http://host.robots.ox.ac.uk/pascal/VOC/databases.html#VOC2005_1), que você já deve ter baixado. Se não baixou ainda, o download não é pequeno (465Mb) então deixe baixando e prossiga com o roteiro. 


## Pesquisa

1. Detectores de pontos de interesse são classificados, basicamente, em três tipos. Quais são eles e quais características locais das imagens são exploradas?

2. O OpenCV possui diversos algoritmos para detecção e descrição de pontos de interesse. Faça uma lista deles abaixo (deixando bem claro qual método faz detecção, extração ou ambos). Coloque o nome do método, a classe que o implementa e seu tipo (para os detectores somente).

3. Escolha um dos métodos acima e descreva sua ideia básica no campo abaixo. Você deve indicar em qual artigo ele foi proposto, quais suas características marcantes em relação aos outros métodos e uma aplicação em que este descritor obteve resultados relavantes. 

## Implementação

Vamos agora implementar o método de similaridade de imagens descrito na aula expositiva, resumido abaixo. Você receberá como entrada uma imagem de busca $Q$ e um banco de imagens $\{F^i\}_{i=0}^N$ contendo $N$ imagens.

1. Detecte os pontos de interesse e extraia os descritores $\{q_j\}$ de $Q$
2. Para cada imagem $F^i$:
    - Detecte os pontos de interesse e extraia os descritores $\{F_k\}$ de $F^i$ 
    - Para cada descritor $q_j$, verifique se existe um descritor $F_k$ que casa com ele
    - Calcule a proporção de descritores de $Q$ que casaram com descritores de $F^i$ e use este valor como a similaridade entre as duas imagens.
    
3. Retorne as 5 imagens com maior similaridade;

### Exercício

Antes de continuar, leia com atenção o algoritmo acima e pense: como você organizaria seu código para implementá-lo? Quais funções auxiliares criaria e quais argumentos cada uma receberia? Supondo que todas essas funções auxiliares já foram implementadas, como seria o código principal de seu programa?

### Extração e descrição de pontos de interesse

A detecção e descrição de pontos de interesse é um processo que será repetido várias vezes nesta e na próxima atividade. Na parte de pesquisa você identificou alguns métodos de extração e descrição de pontos de interesse. Selecione um de cada tipo (ou um que faça ambos) e crie uma função que recebe uma imagem e retorna os descritores dela. Mesmo que a função seja muito pequena (duas ou três linhas), é uma boa fazê-lo para deixar o programa principal mais legível e parecido com o algoritmo descrito em linguagem natural. 

OBS: os exercícios foram testados usando o descritor *ORB*. Você pode usar outro nos seus testes, mas pode ter que adaptar parte do código. 

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
def calc_dist(list1, list2):
    dist = 0
    string1 = '{}'.format(list1)
    string2 = '{}'.format(list2)
    for j in range(len(string1)):
        if(string1[j] != string2[j]):
            dist +=1
    return dist
    

def reciprocal_match(desc1, desc2):
    matches = []
    for i in range (len(desc1)):
        min_dist = calc_dist(desc1[i], desc2[0])
        index2 = 0
        for j in range(1, len(desc2)):
            min_actual = calc_dist(desc1[i], desc2[j])
            if(min_actual < min_dist):
                min_dist = min_actual
                index2 = j
        min_dist2 = calc_dist(desc1[0], desc2[index2])
        index1 = 0
        for k in range (len(desc1)):
            min_actual = calc_dist(desc2[index2], desc1[k])
            if(min_actual < min_dist2):
                min_dist2 = min_actual
                index1 = k
        if(index1 == i):
            matches.append(cv.DMatch(index1, index2, min_dist))
    return matches


def find_features(img):
    # Initiate ORB detector
    orb = cv.ORB_create(nfeatures=50)
    # find the keypoints with ORB
    kp = orb.detect(img,None)
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    return kp, des

def compare_descriptors(qj, Fk):
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(qj,Fk)
    return matches

def calculate_proportion(list_d1, list_d2):    
    matches = compare_descriptors(list_d1, list_d2)
    return len(matches)/len(list_d1)

def compare_images(img1, img2):
    kp1, actual_descriptor = find_features(img1)
    kp2, actual_dataset_descriptor = find_features(img2)
    prop = calculate_proportion(actual_descriptor, actual_dataset_descriptor)
    print(prop)
    
    


Teste sua função usando a imagem *cachorro.jpg*. Você pode visualilzar os pontos de interesse usando a função [`cv2.drawKeypoints()`](https://docs.opencv.org/3.4.3/d4/d5d/group__features2d__draw.html#gab958f8900dd10f14316521c149a60433).

In [ ]:
import matplotlib.pyplot as plt
img = cv.imread("cachorro.jpg")
kp, des = find_features(img)
img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB);
plt.imshow(img2)


### Casamento de pontos de interesse

O OpenCV possui duas técnicas de casamento de pontos de interesse implementadas: `cv2.BFMatcher` e `cv2.FlannBasedMatcher`. Para exercitar a compreensão do algoritmo de casamento vamos implementar nossa própria versão do `BFMatcher`. 

----------

Sim, no projeto vocês podem usar as implementações do OpenCV. Na próxima aula iremos usar o `FlannBasedMatcher` e usar um novo critério para casamento de pontos. 

----------

#### Exercício:

Implemente uma função `reciprocal_math(desc1, desc2)` que faz o casamento dos descritores passados como argumento. Um par de descritores $(d^1_i, d^2_j)$ *casa* se $dist(d^1_i, d^2_j) < dist(d^1_k, d^2_j), k\neq i$ **E** $dist(d^1_i, d^2_j) < dist(d^1_i, d^2_k), k\neq j$. Como função de distância tem duas opções:

1. Se os dados computados pelo seu descritor forem um vetor de float você pode usar a norma $\ell_2$ da diferença dos vetores

$$
dist(d^1_i, d^2_j) = ||d^1_i - d^2_j||_2
$$

2. Se os dados computados forem um vetor de bits você deve usar como distância o número de bits diferentes entre as duas strings. Isto pode ser feito, em Python, em três passos:
    1. Vetores de bits são representados por vetores de inteiros de 8 bits. Para cada um dos inteiros:
    2. Converta-o para uma string contendo sua representação em binário usando a função `format`.
    3. Conte o número de caracteres diferentes nas strings

Para que nossa função `reciprocal_match` seja integrada ao OpenCV precisamos retornar uma lista de objetos do tipo `cv2.DMatch`. Para criar um objeto deste tipo basta chamar o construtor com três parâmetros:

1. índice $i$ do descritor da imagem de pesquisa
2. índice $j$ do descritor da imagem do banco
3. distância calculada como a fórmula acima

In [ ]:
import numpy as np

def find_features(img):
    # Initiate ORB detector
    orb = cv.ORB_create()
    # find the keypoints with ORB
    kp = orb.detect(img,None)
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    return kp, des

def calc_dist(list1, list2):
    dist = 0
    for i in range(len(list1)):
        string1 = '{:08b}'.format(list1[i])
        string2 = '{:08b}'.format(list2[i])
        for j in range(len(string1)):
            if(string1[j] != string2[j]):
                dist +=1
    return dist
    

def reciprocal_match(desc1, desc2):
    matches = []
    for i in range (len(desc1)):
        min_dist = calc_dist(desc1[i], desc2[0])
        index2 = 0
        for j in range(1, len(desc2)):
            min_actual = calc_dist(desc1[i], desc2[j])
            if(min_actual < min_dist):
                min_dist = min_actual
                index2 = j
        min_dist2 = calc_dist(desc1[0], desc2[index2])
        index1 = 0
        for k in range (len(desc1)):
            min_actual = calc_dist(desc2[index2], desc1[k])
            if(min_actual < min_dist2):
                min_dist2 = min_actual
                index1 = k
        if(index1 == i):
            matches.append(cv.DMatch(index1, index2, min_dist))
    return matches

def avg_matcher(matches):
    if(len(matches) != 0):
        print(f"Média: {sum([matcher.distance for matcher in matches])/len(matches)}")
    v = [matcher.distance for matcher in matches]
    print(f"Média 100: {np.average(np.sort(v)[0:100])}")

Vamos analisar os resultados dessa função de casamento usando as imagens *cachorro2.png* e *cidade.jpeg*. Extraia os descritores de ambas e faça um casamento com os descritores extraídos no exercício anterior. Para mostrar os resultados você pode usar a função [`cv2.DrawMatches()`](https://docs.opencv.org/3.4.3/d4/d5d/group__features2d__draw.html#ga7421b3941617d7267e3f2311582f49e1). Mostre também a distância média dos casamentos encontrados. Para comparar, mostre a distância média dos 100 casamentos com menor distância para cada imagem.

In [ ]:
img1 = cv.imread("cachorro.jpg")
img2 = cv.imread("cachorro2.jpg")
kp1, actual_descriptor = find_features(img1)
kp2, actual_dataset_descriptor = find_features(img2)

matches = reciprocal_match(actual_descriptor, actual_dataset_descriptor)
out = cv.drawMatches(img1, kp1, img2, kp2, matches, None)
avg_matcher(matches)
plt.imshow(out)

plt.show()

img1 = cv.imread("cachorro.jpg")
img2 = cv.imread("cidade.jpeg")
kp1, actual_descriptor = find_features(img1)
kp2, actual_dataset_descriptor = find_features(img2)

matches = reciprocal_match(actual_descriptor, actual_dataset_descriptor)
out = cv.drawMatches(img1, kp1, img2, kp2, matches, None)
avg_matcher(matches)
plt.imshow(out)


plt.show()

Em ambos os casos houveram muitos casamentos, mas a distância média foi (deveria ser) menor no caso do cachorro. Modifique a função `reciprocal_match` para receber um terceiro argumento que representa a distância máxima entre descritores para que o casamento seja válido. Determine, então, um valor que torne o número de descritores encontrados nas imagens acima significativamente diferente. 

In [ ]:
def reciprocal_match2(desc1, desc2, max_dist):
    matches = []
    for i in range (len(desc1)):
        min_dist = calc_dist(desc1[i], desc2[0])
        index2 = 0
        for j in range(1, len(desc2)):
            min_actual = calc_dist(desc1[i], desc2[j])
            if(min_actual < min_dist):
                min_dist = min_actual
                index2 = j
        min_dist2 = calc_dist(desc1[0], desc2[index2])
        index1 = 0
        for k in range (len(desc1)):
            min_actual = calc_dist(desc2[index2], desc1[k])
            if(min_actual < min_dist2):
                min_dist2 = min_actual
                index1 = k
        if(index1 == i):
            if(min_dist <= max_dist):
                matches.append(cv.DMatch(index1, index2, min_dist))
    return matches

Teste o valor determinado com a imagem *cachorro3.jpg*. Comente os resultados obtidos.

In [ ]:
img1 = cv.imread("cachorro3.jpg")
img2 = cv.imread("cachorro.jpg")
kp1, actual_descriptor = find_features(img1)
kp2, actual_dataset_descriptor = find_features(img2)

matches = reciprocal_match2(actual_descriptor, actual_dataset_descriptor, 55)
out = cv.drawMatches(img1, kp1, img2, kp2, matches, None)
avg_matcher(matches)
plt.imshow(out)

plt.show()

### Juntando tudo

Use as funções acima para calcular a similaridade entre duas imagens, definida como a proporção de pontos casados em relação ao número de pontos da imagem de pesquisa $Q$. Sua função deve se chamar `similarity_proportion_matches` e deve receber duas imagens como entrada e retornar um número entre $0$ (totalmente diferente) e $1$ (idênticas). 

In [ ]:
def calculate_proportion(list_d1, list_d2):    
    matches = reciprocal_match2(list_d1, list_d2, 55)
    return len(matches), len(list_d1)

def similarity_proportion_matches(img1, img2):
    kp1, actual_descriptor = find_features(img1)
    kp2, actual_dataset_descriptor = find_features(img2)
    prop, l = calculate_proportion(actual_descriptor, actual_dataset_descriptor)
    print(prop/l)

Rode sua função com as imagens usadas no exercício anterior e verifique se sua função de similaridade ordenaria as imagens de maneira satisfatória.

In [ ]:
print("Cachorro: cachorro")
img1 = cv.imread("cachorro.jpg")
# img2 = cv.imread("cidade.jpeg")
img2 = cv.imread("cachorro.jpg")
similarity_proportion_matches(img1, img2)
print()

print("Cachorro: cachorro2")
img1 = cv.imread("cachorro.jpg")
# img2 = cv.imread("cidade.jpeg")
img2 = cv.imread("cachorro2.jpg")
similarity_proportion_matches(img1, img2)
print()

print("Cachorro: cachorro3")
img1 = cv.imread("cachorro.jpg")
# img2 = cv.imread("cidade.jpeg")
img2 = cv.imread("cachorro3.jpg")
similarity_proportion_matches(img1, img2)
print()

print("Cachorro: cidade")
img1 = cv.imread("cachorro.jpg")
# img2 = cv.imread("cidade.jpeg")
img2 = cv.imread("cidade.jpeg")
similarity_proportion_matches(img1, img2)
print()



### Testes e avaliação crítica dos resultados

Você deve agora executar seu código nas imagens da base de dados sugerida no começo do roteiro. São presentes apenas imagens com 4 objetos diferentes: pessoas, motos, bicicletas e carros. Selecione uma imagem de cada tipo e mostre as 3 imagens com maior similaridade para cada uma delas.

In [ ]:
import os
import random 


PATH = '../BVW/PNGImages/'

dirs = os.listdir(PATH)

DATASET = []
for directory in dirs:
    if('.' not in directory):
        actual_path = PATH + directory + '/'
        dir_images = os.listdir(actual_path)
        for path_images in dir_images:
            total_path = actual_path + path_images
#             print(total_path)
            DATASET.append(total_path)

def calculate_proportion(list_d1, list_d2):    
    matches = reciprocal_match2(list_d1, list_d2, 55)
    return len(matches)/len(list_d1)

def compare_all_images(img, dataset):
    prop_dict = {}
    kp, actual_descriptor = find_features(img)
    for image_path in dataset:
        print(image_path)
        image = cv.imread(image_path)
        kp, actual_dataset_descriptor = find_features(image)
        prop = calculate_proportion(actual_descriptor, actual_dataset_descriptor)
        prop_dict[image_path] = prop
    sorted_dict = [(k, prop_dict[k]) for k in sorted(prop_dict, key=prop_dict.get, reverse=True)]
    return sorted_dict

def display_5better(d):
    for i in range(5):
        print(f"{d[i][0]} : {(d[i][1] * 100)}%")
        image = plt.imread(d[i][0])
        plt.imshow(image)
        plt.show()
random.shuffle(DATASET)

In [ ]:
random.shuffle(DATASET)
img1 = cv.imread("./../BVW/PNGImages/Caltech_cars/image_0007.png")
d = compare_all_images(img1, DATASET[0:5])
display_5better(d)

In [ ]:
random.shuffle(DATASET)
img1 = cv.imread("../BVW/PNGImages/Caltech_motorbikes_side/0163.png")
d = compare_all_images(img1, DATASET[0:5])
display_5better(d)

In [ ]:
random.shuffle(DATASET)
img1 = cv.imread("../BVW/PNGImages/TUGraz_bike/bike_184.png")
d = compare_all_images(img1, DATASET[0:5])
display_5better(d)

*Meus comentários sobre os resultados aqui*

Faça agora uma busca usando uma imagem focada em um outro objeto qualquer e mostre as 3 imagens mais similares abaixo. 

In [ ]:
random.shuffle(DATASET)
img1 = cv.imread("../aula_17/101_ObjectCategories/gerenuk/image_0003.jpg")
d = compare_all_images(img1, DATASET[0:5])
display_5better(d)

*Meus comentários sobre os resultados aqui*

### Análise crítica e Revisão de conceitos

Descreva com suas próprias palavras o quê são pontos de interesse e descritores.  

Pontos de interesse são pontos que diferem de grandes regiões comuns da imagem. Normalmente são bordas, cantos ou regiões com texturas.

Descritores são objetos que descrevem características da vizinhança de um ponto de interesse da imagem.

Construimos uma abordagem que ordenou as três imagens testadas. Comente uma desvantagem desta abordagem. 

Uma desvantagem desta abordagem se encontra no fato de existir apenas um casamento por keypoint ou pela distância máxima precisar ser calibrada depois posteriormente.